In [1]:
import os
import pandas as pd
import numpy as np
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split

import tensorflow as tf
from sklearn.model_selection import train_test_split

print(tf.config.list_physical_devices('GPU'))

[]


In [2]:
# The datasets are available in CoCalc in ~/share/data/I-coopetition-muon-id/
# Test
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/dd6255a1-a14b-4276-9a2b-db7f360e01c7
# Train
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/3a5e940c-2382-4716-9ff7-8fbc269b98ac

In [3]:
# columns = utils.SIMPLE_FEATURE_COLUMNS + ["id", "label", "weight", "sWeight", "kinWeight"]
# # DATA_PATH = "."
# DATA_PATH = "~/share/data/I-coopetition-muon-id/"
# train = pd.read_csv(os.path.join(DATA_PATH, "train.csv.gz"), index_col="id", usecols=columns)
# test = pd.read_csv(os.path.join(DATA_PATH, "test-features.csv.gz"), index_col="id", usecols=utils.SIMPLE_FEATURE_COLUMNS + ["id"])

In [4]:
columns = utils.SIMPLE_FEATURE_COLUMNS + ["id", "label", "weight",  "kinWeight"]
DATA_FOLDER = "~/share/data/1.6.2-boosting/"
data = pd.read_csv(os.path.join(DATA_FOLDER, "train_1_percent.csv"), index_col="id", usecols=columns)


new_key_label=[]
for name in data.columns:
    if '[' in name:
        name = name.replace('[', '_').replace(']', '')

    new_key_label.append(name)

data.columns = new_key_label


        

In [5]:
data.head(5)

,ncl_0,ncl_1,ncl_2,ncl_3,avg_cs_0,avg_cs_1,avg_cs_2,avg_cs_3,ndof,MatchedHit_TYPE_0,...,Mextra_DY2_0,Mextra_DY2_1,Mextra_DY2_2,Mextra_DY2_3,FOI_hits_N,PT,P,label,kinWeight,weight
id,,,,,,,,,,,,,,,,,,,,,
156079,24,2,11,13,2.333333,2.000000,1.727273,1.153846,8,2,...,609.451230,1758.819800,4016.10500,7918.94200,7,1699.454257,8692.546221,1,2.80650,2.431642
1129839,24,13,6,10,1.708333,1.769231,1.500000,1.400000,8,2,...,342.040680,989.623600,2251.83450,4411.09700,5,3381.439699,13378.001174,1,1.85646,2.259009
4263309,27,13,10,13,2.296296,1.384615,1.100000,2.000000,8,2,...,256.180200,744.244750,1684.75510,3292.70750,12,962.785246,13904.582780,1,3.77936,4.349500
199338,130,66,23,23,3.423077,3.045454,1.434783,1.565217,8,2,...,649.849800,1885.884300,4325.38430,8559.56000,64,810.190921,8565.752002,1,1.85646,-0.740073
2663246,85,61,18,20,3.294118,2.278688,1.222222,1.300000,8,2,...,12.617317,36.086983,79.09466,148.79881,10,1452.814360,64736.044173,1,1.85646,1.161770


In [6]:
#prepare samples
train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(data), 'data examples')
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

batch_size = 32
shuffle = True
train_ds = utils.df_to_dataset(train, batch_size=batch_size)
val_ds = utils.df_to_dataset(val, shuffle=shuffle, batch_size=batch_size)
test_ds = utils.df_to_dataset(test, shuffle=shuffle, batch_size=batch_size)

train_labels = np.array(train.pop('label'))
val_labels = np.array(val.pop('label'))
test_labels = np.array(test.pop('label'))

print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)
# print(train_ds)

train_features = np.array(train)
val_features = np.array(val)
test_features = np.array(test)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)
print('Test features shape:', train_labels)



12637 data examples
8087 train examples
2022 validation examples
2528 test examples


Training labels shape: (8087,)
Validation labels shape: (2022,)
Test labels shape: (2528,)
Training features shape: (8087, 67)
Validation features shape: (2022, 67)
Test features shape: (2528, 67)
Test features shape: [0 1 1 ... 0 1 0]


In [7]:
# for feature_batch, label_batch in train_ds.take(1):
#   print('Every feature:', list(feature_batch.keys()))
#   print('A batch of PT:', feature_batch['PT'])
#   print('A batch of targets:', label_batch )


In [8]:
# test.head()

In [13]:
# numeric cols
feature_columns = []
feature_batch, label_batch = next(iter(train_ds))
# for feature_batch, label_batch in train_ds.take(1):
for header in list(feature_batch.keys()):
    feature_columns.append(tf.feature_column.numeric_column(header))

# print(feature_batch)
# print(feature_columns)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns, dtype='float64')
print(feature_layer)

In [169]:
model = tf.keras.Sequential([
  feature_layer,
  tf.keras.layers.Dense(128, activation='relu', name="layer1" ),
  tf.keras.layers.Dense(128, activation='relu', name="layer2"),
  tf.keras.layers.Dropout(.1, name="dropout1"),
  tf.keras.layers.Dense(1, name="layer3")
])


model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [170]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)


Epoch 1/10


  1/506 [..............................] - ETA: 0s - loss: 523.6953 - accuracy: 0.3438

 16/506 [..............................] - ETA: 1s - loss: 717.0784 - accuracy: 0.5566

 31/506 [>.............................] - ETA: 1s - loss: 562.4260 - accuracy: 0.5756

 46/506 [=>............................] - ETA: 1s - loss: 504.5508 - accuracy: 0.5652

...

343/506 [===================>..........] - ETA: 0s - loss: 0.7595 - accuracy: 0.6861

358/506 [====================>.........] - ETA: 0s - loss: 0.7567 - accuracy: 0.6856

373/506 [=====================>........] - ETA: 0s - loss: 0.7532 - accuracy: 0.6843

388/506 [======================>.......] - ETA: 0s - loss: 0.8651 - accuracy: 0.6844

403/506 [======================>.......] - ETA: 0s - loss: 0.8567 - accuracy: 0.6839

In [150]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
print(len(val), 'validation examples')
validation_predictions = probability_model.predict(val_ds)
# print(validation_predictions)
validation_predictions.shape


2022 validation examples


(4044, 1)

In [65]:
scoring.rejection90(val_ds.labels, validation_predictions, sample_weight=validation.weight.values)

AttributeError: 'BatchDataset' object has no attribute 'labels'

In [11]:
model.fit(train.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.kinWeight.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
predictions = model.predict_proba(test.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [16]:
# compression_opts = dict(method='zip',
#                         archive_name='submission.csv')  
# pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
#     "submission.zip", index_label=utils.ID_COLUMN, compression=compression_opts)